# EDA for Combined Tables: Topics and Bills

In [1]:
import mysql.connector 
import numpy as np
import pandas as pd
import config_final
import requests
from utilities import query, query_list


from sodapy import Socrata
import sqlalchemy as db


import config_final as config
from schema import DbSchema

import pickle


In [2]:
bills_db = DbSchema(config)

**Connect to Database**

In [3]:
#Query all titles and Passing

df = bills_db.query("""
    SELECT
        *
    FROM con_bills.current_bills as cb
    JOIN con_bills.topics as tp
    ON cb.BillID = tp.BillID
    WHERE cb.Cong >=110
    """)
df.head()

,BillID,BillType,Chamber,Cong,Cosponsr,IntrDate,Mult,PLaw,Title,Veto,...,PLawNum,ImpBill,Majority,Major,Minor,URL,Summary,Index,BillID,dominant_topic
0,110-HR-1,hr,0,110,205.0,2007-01-05 00:00:00,1,1,To provide for the implementation of the recom...,0,...,Became Public Law No: 110-53.,1,1,16.0,1615.0,https://www.congress.gov/bill/110th-congress/h...,Implementing Recommendations of the 9/11 Commi...,51068,110-HR-1,5
1,110-HR-10,hr,0,110,0.0,2007-01-04 00:00:00,0,0,Reserved for Speaker.,0,...,0,1,1,20.0,2002.0,https://www.congress.gov/bill/110th-congress/h...,,51069,110-HR-10,10
2,110-HR-100,hr,0,110,10.0,2007-01-04 00:00:00,0,0,To amend the Higher Education Act of 1965 to p...,0,...,0,1,1,6.0,601.0,https://www.congress.gov/bill/110th-congress/h...,Veterans' Equity in Education Act of 2007 - Am...,51070,110-HR-100,4
3,110-HR-1000,hr,0,110,286.0,2007-02-12 00:00:00,0,0,To award a congressional gold medal to Edward ...,0,...,0,0,1,20.0,2006.0,https://www.congress.gov/bill/110th-congress/h...,Edward William Brooke III Congressional Gold M...,51071,110-HR-1000,10
4,110-HR-1001,hr,0,110,0.0,2007-02-12 00:00:00,0,0,To amend the Haitian Hemispheric Opportunity t...,0,...,0,1,1,19.0,1921.0,https://www.congress.gov/bill/110th-congress/h...,Amends the Caribbean Basin Economic Recovery A...,51072,110-HR-1001,5


**Review All data**

Important coding:
    
- Party: (100 Democrat; 200 Republican, 328…from ICPSR)
- Cosponsor: Number of cosponsors
    

In [4]:
df.head()

,BillID,BillType,Chamber,Cong,Cosponsr,IntrDate,Mult,PLaw,Title,Veto,...,PLawNum,ImpBill,Majority,Major,Minor,URL,Summary,Index,BillID,dominant_topic
0,110-HR-1,hr,0,110,205.0,2007-01-05 00:00:00,1,1,To provide for the implementation of the recom...,0,...,Became Public Law No: 110-53.,1,1,16.0,1615.0,https://www.congress.gov/bill/110th-congress/h...,Implementing Recommendations of the 9/11 Commi...,51068,110-HR-1,5
1,110-HR-10,hr,0,110,0.0,2007-01-04 00:00:00,0,0,Reserved for Speaker.,0,...,0,1,1,20.0,2002.0,https://www.congress.gov/bill/110th-congress/h...,,51069,110-HR-10,10
2,110-HR-100,hr,0,110,10.0,2007-01-04 00:00:00,0,0,To amend the Higher Education Act of 1965 to p...,0,...,0,1,1,6.0,601.0,https://www.congress.gov/bill/110th-congress/h...,Veterans' Equity in Education Act of 2007 - Am...,51070,110-HR-100,4
3,110-HR-1000,hr,0,110,286.0,2007-02-12 00:00:00,0,0,To award a congressional gold medal to Edward ...,0,...,0,0,1,20.0,2006.0,https://www.congress.gov/bill/110th-congress/h...,Edward William Brooke III Congressional Gold M...,51071,110-HR-1000,10
4,110-HR-1001,hr,0,110,0.0,2007-02-12 00:00:00,0,0,To amend the Haitian Hemispheric Opportunity t...,0,...,0,1,1,19.0,1921.0,https://www.congress.gov/bill/110th-congress/h...,Amends the Caribbean Basin Economic Recovery A...,51072,110-HR-1001,5


In [5]:
df.shape

(51067, 32)

In [6]:
#Review the dependent variable counts
df['PassH'].value_counts()

0    47042
1     4025
Name: PassH, dtype: int64

In [7]:
df['Party'].value_counts()

100.0    30400
200.0    20660
328.0        7
Name: Party, dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51067 entries, 0 to 51066
Data columns (total 32 columns):
BillID            51067 non-null object
BillType          51067 non-null object
Chamber           51067 non-null int64
Cong              51067 non-null int64
Cosponsr          51067 non-null float64
IntrDate          51025 non-null object
Mult              51067 non-null int64
PLaw              51067 non-null int64
Title             51067 non-null object
Veto              51067 non-null int64
Class             51067 non-null float64
District          51067 non-null float64
FrstConH          51067 non-null float64
FrstConS          51067 non-null float64
Gender            51067 non-null int64
MRef              51067 non-null int64
NameFull          51067 non-null object
Party             51067 non-null float64
Postal            51067 non-null object
PassH             51067 non-null int64
PassS             51067 non-null int64
PLawDate          1393 non-null object
PLawNum        

**Combined Text:**

In [9]:
df['combined_text'] = df[['Title', 'Summary']].astype(str).apply(' '.join, axis=1)

In [10]:
df.head()

,BillID,BillType,Chamber,Cong,Cosponsr,IntrDate,Mult,PLaw,Title,Veto,...,ImpBill,Majority,Major,Minor,URL,Summary,Index,BillID,dominant_topic,combined_text
0,110-HR-1,hr,0,110,205.0,2007-01-05 00:00:00,1,1,To provide for the implementation of the recom...,0,...,1,1,16.0,1615.0,https://www.congress.gov/bill/110th-congress/h...,Implementing Recommendations of the 9/11 Commi...,51068,110-HR-1,5,To provide for the implementation of the recom...
1,110-HR-10,hr,0,110,0.0,2007-01-04 00:00:00,0,0,Reserved for Speaker.,0,...,1,1,20.0,2002.0,https://www.congress.gov/bill/110th-congress/h...,,51069,110-HR-10,10,Reserved for Speaker.
2,110-HR-100,hr,0,110,10.0,2007-01-04 00:00:00,0,0,To amend the Higher Education Act of 1965 to p...,0,...,1,1,6.0,601.0,https://www.congress.gov/bill/110th-congress/h...,Veterans' Equity in Education Act of 2007 - Am...,51070,110-HR-100,4,To amend the Higher Education Act of 1965 to p...
3,110-HR-1000,hr,0,110,286.0,2007-02-12 00:00:00,0,0,To award a congressional gold medal to Edward ...,0,...,0,1,20.0,2006.0,https://www.congress.gov/bill/110th-congress/h...,Edward William Brooke III Congressional Gold M...,51071,110-HR-1000,10,To award a congressional gold medal to Edward ...
4,110-HR-1001,hr,0,110,0.0,2007-02-12 00:00:00,0,0,To amend the Haitian Hemispheric Opportunity t...,0,...,1,1,19.0,1921.0,https://www.congress.gov/bill/110th-congress/h...,Amends the Caribbean Basin Economic Recovery A...,51072,110-HR-1001,5,To amend the Haitian Hemispheric Opportunity t...


In [11]:
df.iloc[555][8]
df.iloc[555][28]

'Short Sea Shipping Promotion Act of 2007 - Amends the Internal Revenue Code to exempt from the harbor maintenance tax cargo contained in intermodal cargo containers and loaded by crane on a vessel, or cargo loaded on a vessel by means of wheeled technology, that is: (1) loaded at a U.S. port and unloaded at another U.S. port or a port in Canada located in the Great Lakes Saint Lawrence Seaway System; or (2) loaded at a port in Canada located in the Great Lakes Saint Lawrence Seaway System and unloaded at a U.S. port.'

In [12]:
new = df[['PassH', 'combined_text', 'Cong', 'BillID']].copy()

In [13]:
new.head()

,PassH,combined_text,Cong,BillID,BillID
0,1,To provide for the implementation of the recom...,110,110-HR-1,110-HR-1
1,0,Reserved for Speaker.,110,110-HR-10,110-HR-10
2,0,To amend the Higher Education Act of 1965 to p...,110,110-HR-100,110-HR-100
3,0,To award a congressional gold medal to Edward ...,110,110-HR-1000,110-HR-1000
4,0,To amend the Haitian Hemispheric Opportunity t...,110,110-HR-1001,110-HR-1001


In [14]:
passed_only = new.loc[new['PassH'] != 0] 

print(len(passed_only))
passed_only.head()

4025


,PassH,combined_text,Cong,BillID,BillID
0,1,To provide for the implementation of the recom...,110,110-HR-1,110-HR-1
6,1,To amend the Foreign Affairs Reform and Restru...,110,110-HR-1003,110-HR-1003
9,1,To amend the provisions of law relating to the...,110,110-HR-1006,110-HR-1006
15,1,To designate additional National Forest System...,110,110-HR-1011,110-HR-1011
18,1,"To amend the Federal Food, Drug, and Cosmetic ...",110,110-HR-1014,110-HR-1014


In [15]:
passed_only.iloc[6][1]

'To direct the Secretary of the Interior to conduct a special resources study regarding the suitability and feasibility of designating certain historic buildings and areas in Taunton, Massachusetts, as a unit of the National Park System, and for other purposes. (This measure has not been amended since it was introduced. The summary has been expanded because action occurred on the measure.)'

**Basic NLP EDA:**

In [16]:
df['Summary'].isnull().sum()

50

In [17]:
df['text_len'] = [len(summary) for summary in df['combined_text']]

In [18]:
df.head()

,BillID,BillType,Chamber,Cong,Cosponsr,IntrDate,Mult,PLaw,Title,Veto,...,Majority,Major,Minor,URL,Summary,Index,BillID,dominant_topic,combined_text,text_len
0,110-HR-1,hr,0,110,205.0,2007-01-05 00:00:00,1,1,To provide for the implementation of the recom...,0,...,1,16.0,1615.0,https://www.congress.gov/bill/110th-congress/h...,Implementing Recommendations of the 9/11 Commi...,51068,110-HR-1,5,To provide for the implementation of the recom...,331
1,110-HR-10,hr,0,110,0.0,2007-01-04 00:00:00,0,0,Reserved for Speaker.,0,...,1,20.0,2002.0,https://www.congress.gov/bill/110th-congress/h...,,51069,110-HR-10,10,Reserved for Speaker.,22
2,110-HR-100,hr,0,110,10.0,2007-01-04 00:00:00,0,0,To amend the Higher Education Act of 1965 to p...,0,...,1,6.0,601.0,https://www.congress.gov/bill/110th-congress/h...,Veterans' Equity in Education Act of 2007 - Am...,51070,110-HR-100,4,To amend the Higher Education Act of 1965 to p...,720
3,110-HR-1000,hr,0,110,286.0,2007-02-12 00:00:00,0,0,To award a congressional gold medal to Edward ...,0,...,1,20.0,2006.0,https://www.congress.gov/bill/110th-congress/h...,Edward William Brooke III Congressional Gold M...,51071,110-HR-1000,10,To award a congressional gold medal to Edward ...,400
4,110-HR-1001,hr,0,110,0.0,2007-02-12 00:00:00,0,0,To amend the Haitian Hemispheric Opportunity t...,0,...,1,19.0,1921.0,https://www.congress.gov/bill/110th-congress/h...,Amends the Caribbean Basin Economic Recovery A...,51072,110-HR-1001,5,To amend the Haitian Hemispheric Opportunity t...,575


In [19]:
df.groupby(['PassH','Cong'])['text_len'].agg('median')

PassH  Cong
0      110     515.0
       111     527.0
       112     487.0
       113     554.0
       114     223.0
1      110     450.0
       111     380.0
       112     355.0
       113     345.5
       114     278.0
Name: text_len, dtype: float64

In [20]:
#Median text length by congress, passed or didn't pass
df.groupby(['PLaw','Cong'])['text_len'].agg('median')

PLaw  Cong
0     110     514.0
      111     525.0
      112     486.0
      113     550.0
      114     226.0
1     110     395.5
      111     329.5
      112     310.5
      113     300.0
      114     299.0
Name: text_len, dtype: float64

In [21]:
df['title_len'] = [len(summary) for summary in df['Title']]

In [22]:
df['Summary'].fillna('', inplace=True)

/Users/melissamunz/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [23]:
df['summary_len'] = [len(summary) for summary in df['Summary']]

TypeError: object of type 'NoneType' has no len()

In [ ]:
#Median Bill Title length by congress, passed or didn't pass
df.groupby(['PLaw','Cong'])['title_len'].agg('median')

**Basic Feature Info:**

In [ ]:
df['Cong'].value_counts()

In [ ]:
df['Cong'].value_counts()

In [ ]:
df['BillType'].value_counts()

**General Visualizations:**

In [ ]:
import seaborn as sns
sns.set(style="whitegrid")

df['PassH'].value_counts().plot.barh().set_title('Passed vs. Proposed Bills')

In [ ]:
df['Chamber'].value_counts().plot.bar().set_title('Bills Proposed by Chamber')

In [ ]:
df['Cong'].value_counts().plot.barh().set_title('Number of Bills by Recent Congress')


In [ ]:
#Bills Proposed by Majority or Not, for all Congresses
#DONT USE< DOESNT REALLY MAKE SENSE

df.groupby(['Cong', 'Majority']).size().unstack().plot.bar(stacked=True).set_title('Majority vs Minority House Bill Proposals')


In [ ]:
#Portion of bills proposed for all congresses vs those that passed the house 
df.groupby(['Cong', 'PassH']).size().unstack().plot.bar(stacked=True).set_title('Bill Passage in House per Congress')


In [ ]:
#Portion of bills proposed for all congresses vs those that passed the house 
df.groupby(['Cong', 'PassS']).size().unstack().plot.bar(stacked=True).set_title('Bill Passage Senate per Congress')


In [ ]:
#Portion of bills proposed by the House and Senate
#0 for House, 1 for Senate
df.groupby(['Cong', 'Chamber']).size().plot.bar(stacked=True).set_title('Bill Proposal by Chamber')


In [ ]:
#All bills proposed in each congress, by Gender

df.groupby(['Cong', 'Gender']).size().unstack().plot.bar(stacked=True).set_title('Bill Proposal by Gender')


In [ ]:
df.groupby(['Cong', 'Majority', 'PassH']).size().unstack().plot.bar(stacked=True).set_title('Bill Passage by Majority')


In [ ]:
df.groupby(['Cong', 'Party']).size().unstack().plot.bar(stacked=True).set_title('Bill Proposal by Party')


In [ ]:
df.groupby(['Cong', 'Majority']).size().unstack().plot.bar(stacked=True).set_title('Bill Proposal by Majority')


In [ ]:
#Party code (100 Democrat; 200 Republican, 328…from ICPSR)
df.groupby(['Cong', 'Party']).size().unstack().plot.bar().set_title('Party breakdown by Congress')

In [ ]:
#Proportional Party breakdown by Congress
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

plt.figure(figsize=(20,5))

x, y, hue = "Cong", "proportion", "Party"
hue_order = ["1", "0"]
data=df

(df[x]
 .groupby(df[hue])
 .value_counts(normalize=True)
 .rename(y)
 .reset_index()
 .pipe((sns.barplot, "data"), x=x, y=y, hue=hue))


In [ ]:
#Proportional Party breakdown by Congress
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

plt.figure(figsize=(20,5))

x, y, hue = "Cong", "proportion", "PassH"
hue_order = ["1", "0"]
data=df

(df[x]
 .groupby(df[hue])
 .value_counts(normalize=True)
 .rename(y)
 .reset_index()
 .pipe((sns.barplot, "data"), x=x, y=y, hue=hue)
 .set_title('Proportion Bill Passage in House by Congress'))


**By State:**

.size includes NaN values, .count does not

In [ ]:
#Breakdown of bills proposed by party by state

df.groupby(['Postal', 'Party']).size().unstack().plot.bar(figsize=(20,8), stacked=True).set_title('Party breakdown by State')

In [ ]:
df.groupby(['Postal', 'PassH']).size().unstack().plot.bar(figsize=(20,8), stacked=True).set_title('Bill Passage by State')

In [ ]:
df.groupby(['Postal', 'PassH']).size().unstack().plot.bar(figsize=(20,8), stacked=True).set_title('Bill Passage by State')

In [ ]:
#Passed Laws by state. 0 is Not passed, 1 passed
df.groupby(['Postal', 'PLaw']).size().unstack().plot.bar(figsize=(20,8), stacked=True).set_title('Bill to Law Passage by State')

In [ ]:
plt.figure(figsize=(20,5))
sns.barplot(x='Postal', y='PassH', data=df, estimator=lambda x: sum(x==1 )*100.0/len(x))


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

plt.figure(figsize=(20,5))

x, y, hue = "Postal", "proportion", "PassH"
hue_order = ["1", "0"]
data=df

(df[x]
 .groupby(df[hue])
 .value_counts(normalize=True)
 .rename(y)
 .reset_index()
 .pipe((sns.barplot, "data"), x=x, y=y, hue=hue)).set_title('Title')


In [ ]:
# def compute_percentage(x):
#     pct = float(x/df['PassH'].count()) * 100
#     return round(pct, 2)

# df['state_weight'] = df.apply(compute_percentage, axis=1)
# df['state_weight']=map(lambda x: sum(x==1 )*100.0/len(x))

In [ ]:
import plotly.express as px  # Be sure to import express

fig = px.choropleth(df,  # Input Pandas DataFrame
                    locations="Postal",  # DataFrame column with locations
                    color="value",  # DataFrame column with color values
                    hover_name="Postal", # DataFrame column hover info
                    locationmode = 'USA-states') # Set to plot as US States
fig.update_layout(
    title_text = 'State Rankings', # Create a Title
    geo_scope='usa',  # Plot only the USA instead of globe
)
fig.show()  # Output the plot to the screen

In [ ]:
import plotly.graph_objects as go


# fig = go.Figure(data=go.Choropleth(
#     locations=df['Postal'], # Spatial coordinates
#     z = df['PassH'].astype(float), # Data to be color-coded
#     locationmode = 'USA-states', # set of locations match entries in `locations`
#     colorscale = 'Reds',
#     colorbar_title = "Millions USD",
# ))

# fig.update_layout(
#     title_text = '2011 US Agriculture Exports by State',
#     geo_scope='usa', # limite map scope to USA
# )

# fig.show()

**NLP Graphs:**

In [ ]:
df.groupby(['PassH','Cong'])['text_len'].agg('mean').unstack().plot.bar().set_title('Length of Title/Summary Text by Congress - Not Pass and Pass')

In [ ]:
df.groupby(['PassH','Cong'])['title_len'].agg('median').unstack().plot.bar().set_title('Median Length of Title Text by Congress')

In [ ]:
#find median length of all titles
df['title_len'].median()

In [ ]:
df['summary_len'].median()

**EDA Passed Laws:**

In [ ]:
#Bill vs number of Congresspersons - whip and majority or others?

In [ ]:
df['NameFull'].value_counts().head()

In [ ]:
from collections import Counter

all_names_dict = dict(Counter(df['NameFull']))

In [ ]:
top_names = {key: value for key, value in all_names_dict.items() if value >= 100}


In [ ]:
for name in sorted(top_names, key=top_names.get, reverse = True):
    print(name, top_names[name])

In [ ]:
# The top 50 Congresspersons and number of bill introductions

data['NameFull'].value_counts().head(50).plot(kind='barh', figsize=(20,20))

**Map Graph**

In [ ]:
!pip install folium



In [ ]:
import folium

In [ ]:
state_geo = os.path.join('/Users/y.holtz/Desktop/', 'us-states.json')


**References**

- https://www.drawingfromdata.com/setting-figure-size-using-seaborn-and-matplotlib